In [ ]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import json
import pickle
from collections import defaultdict
import dill

In [ ]:
def find_salary(years, months):
    n_ads = 0
    ids_set = set()
    skill_dict = defaultdict(lambda: defaultdict(int))
    for year in years:
        for month in months: 
            directory = '/Volumes/Elements/year=' + year + '/month=' + month + '/'
            day = glob(directory + '/*')
            for j in tqdm(range(len(day))):
                filename = glob(day[j] + '/*')
                for file in filename:
                    for line in open(file, 'r'):
                        jsonfile = json.loads(line)
                        # deduplicating
                        try:
                            if jsonfile['salary_currency'] == 'GBP':
                                salary = jsonfile['salary_predicted']   
                                skills = jsonfile['skills']
                                job_id = jsonfile['id']
                                # if 'id' seen then pass
                                old_len = len(ids_set)
                                ids_set.add(job_id)
                                if old_len != len(ids_set):                                
                                    # skills = skills[1:-1].split(', ')           # comment out for 04/2022    
                                    for skill in skills:
                                        skill_dict[skill]['salary'] += salary
                                        skill_dict[skill]['freq'] += 1
                        except:
                            pass
    return skill_dict

In [ ]:
sal_16 = find_salary(['2016'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
sal_18 = find_salary(['2018'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
sal_20 = find_salary(['2020'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
sal_221to3 = find_salary(['2022'], ['1', '2', '3'])

In [ ]:
sal_224to12 = find_salary(['2022'], ['4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
dicts_list = [sal_221to3, sal_224to12]
sal_22 = defaultdict(lambda: defaultdict(int))

for d in dicts_list:
    for skill, info in d.items():
        sal_22[skill]['salary'] += info['salary']
        sal_22[skill]['freq'] += info['freq']

In [ ]:
with open('Adzuna/salary_16.pkl', 'wb') as f:
   dill.dump(sal_16, f)
with open('Adzuna/salary_18.pkl', 'wb') as f:
   dill.dump(sal_18, f)
with open('Adzuna/salary_20.pkl', 'wb') as f:
   dill.dump(sal_20, f)
with open('Adzuna/salary_22.pkl', 'wb') as f:
   dill.dump(sal_22, f)

In [ ]:
def find_n_ads(years, months):
    n_ads = 0
    ids_set = set()
    for year in years:
        for month in tqdm(months): 
            directory = '/Volumes/Elements/year=' + year + '/month=' + month + '/'
            day = glob(directory + '/*')
            for j in range(len(day)):
                filename = glob(day[j] + '/*')
                for file in filename:
                    for line in open(file, 'r'):
                        jsonfile = json.loads(line)
                        # deduplicating
                        job_id = jsonfile['id']
                        # if 'id' seen then pass
                        old_len = len(ids_set)
                        ids_set.add(job_id)
                        if old_len != len(ids_set):
                            n_ads += 1
                            
    return n_ads

In [ ]:
find_n_ads(['2018'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
find_n_ads(['2020'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
find_n_ads(['2016'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
find_n_ads(['2022'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
def find_freq(years, months):
    ids_set = set()
    skill_freq = defaultdict(int)
    for year in years:
        for month in months: 
            directory = '/Volumes/Elements/year=' + year + '/month=' + month + '/'
            day = glob(directory + '/*')
            for j in tqdm(range(len(day))):
                filename = glob(day[j] + '/*')
                for file in filename:
                    for line in open(file, 'r'):
                        jsonfile = json.loads(line)
                        # deduplicating
                        job_id = jsonfile['id']
                        # if 'id' seen then pass
                        old_len = len(ids_set)
                        ids_set.add(job_id)
                        try:
                            if old_len != len(ids_set):
                                skills = jsonfile['skills']
                                skills = skills[1:-1].split(', ')           # comment out for 04/2022    
                                for skill in skills:
                                    skill_freq[skill] += 1
                        except:
                            pass
                            
    return skill_freq

In [ ]:
freq_20224to12 = find_freq2022(['2022'], ['4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
freq_20221to3 = find_freq(['2022'], ['1', '2', '3'])

In [ ]:
freq_16 = find_freq(['2016'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
# List of dictionaries
dicts_list = [freq_20221to3, freq_20224to12]
freq_22 = defaultdict(int)

for d in dicts_list:
    for skill, freq in d.items():
        freq_22[skill] += freq

## Delete the skills that are not in the xlsx by dict operations

In [ ]:
skills_in_mat = list(freq_16.keys())

In [ ]:
match_df = pd.read_excel('Adzuna/Adzuna_Lightcast_skills_dictionary_20231018.xlsx', sheet_name = 'Adzuna_Lightcast_skills_diction')
skills_in_xlsx = list(match_df[match_df['Corrected -> unique skill'] != '[drop]']['Adzuna_skills'])

In [ ]:
skills_todrop = [skill for skill in skills_in_mat if skill not in skills_in_xlsx]

In [ ]:
# Delete the specified keys from the defaultdict of defaultdict
for skill in skills_todrop:
    if skill in freq_16:
        del freq_16[skill]

In [ ]:
len(freq_16.keys())

## Convert to Lightcast Taxonomy

In [ ]:
# Extract the two columns as lists
column1_values = match_df[match_df['Corrected -> unique skill'] != '[drop]']['Adzuna_skills'].tolist()
column2_values = match_df[match_df['Corrected -> unique skill'] != '[drop]']['Lightcast_skills'].tolist()

# Create a dictionary where column1 is the key and column2 is the value
semantic_equivalents = dict(zip(column1_values, column2_values))

In [ ]:
# Create a new defaultdict of defaultdict based on semantic equivalents while ensuring symmetry
freq_16lightcast = defaultdict(int)

for element1, freq in freq_16.items():
    element1 = semantic_equivalents.get(element1, element1)
    freq_16lightcast[element1] += freq

In [ ]:
len(freq_16lightcast)

In [ ]:
with open('Adzuna/freq_lightcastskills_16.pkl', 'wb') as f:
   pickle.dump(freq_16lightcast, f)

In [ ]:
freq_lightcast['Change Control']

# Get co-occurrences from raw Adzuna data

In [ ]:
def find_cooccurrence(years, months):
    # create an set for unique ids
    ids_set = set()
    # initiate co-occurrrence matrix
    co_occurrence_counts = defaultdict(lambda: defaultdict(int))
    # loop over the directories
    for year in years:
        for month in months: 
            directory = '/Volumes/Elements/year=' + year + '/month=' + month + '/'
            day = glob(directory + '/*')
            for j in tqdm(range(len(day))):
                filename = glob(day[j] + '/*')
                for file in filename:
                    # loading each line in a json file
                    for line in open(file, 'r'):
                        jsonfile = json.loads(line)
                        # deduplicating
                        job_id = jsonfile['id']
                        # if 'id' seen then pass
                        old_len = len(ids_set)
                        ids_set.add(job_id)
                        # avoid empty job adverts
                        try:
                            if old_len != len(ids_set):
                                skills = jsonfile['skills']
                                # dtype different before/after 04/2022
                                skills = skills[1:-1].split(', ')           # comment out for 04/2022    
                                # storing the co-ocurrences in the symmetric matrix
                                for i in range(len(skills)):
                                    for j in range(i + 1, len(skills)):
                                        skill1, skill2 = skills[i], skills[j]
                                        co_occurrence_counts[skill1][skill2] += 1
                                        co_occurrence_counts[skill2][skill1] += 1
                        except:
                            pass
                            
    return co_occurrence_counts

In [ ]:
co_dict = find_cooccurrence(['2016', '2018', '2020'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
co_dict2022_1to3 = find_cooccurrence(['2022'], ['1', '2', '3'])

In [ ]:
co_dict22_4to12 = find_cooccurrence(['2022'], ['4', '5', '6', '7', '8', '9', '10', '11', '12'])

In [ ]:
def combine_defaultdict(co_dict1, co_dict2):
    co_dict_combined = defaultdict(lambda: defaultdict(int))
    
    # Update the combined dictionary with the counts from the first dictionary
    for element1, inner_dict1 in co_dict1.items():
        for element2, count1 in inner_dict1.items():
            co_dict_combined[element1][element2] += count1
    
    # Update the combined dictionary with the counts from the second dictionary
    for element1, inner_dict2 in co_dict2.items():
        for element2, count2 in inner_dict2.items():
            co_dict_combined[element1][element2] += count2
    
    return co_dict_combined

In [ ]:
co_dict2022 = combine_defaultdict(co_dict22_1to3, co_dict22_4to12)
co_dict_combined = combine_defaultdict(co_dict2022, co_dict161820all)

In [ ]:
match_df = pd.read_excel('Adzuna/Adzuna_Lightcast_skills_dictionary_20231018.xlsx', sheet_name = 'Adzuna_Lightcast_skills_diction')
skills_in_xlsx = list(match_df[match_df['Corrected -> unique skill'] != '[drop]']['Adzuna_skills'])

In [ ]:
skills_todrop = [skill for skill in skills_in_mat if skill not in skills_in_xlsx]

In [ ]:
# Delete the specified keys from the defaultdict of defaultdict
for skill in skills_todrop:
    if skill in co_dict_combined:
        del co_dict_combined[skill]
    for inner_dict in co_dict_combined.values():
        if skill in inner_dict:
            del inner_dict[skill]

In [ ]:
len(co_dict_combined.keys())

In [ ]:
# Extract the two columns as lists
column1_values = match_df[match_df['Corrected -> unique skill'] != '[drop]']['Adzuna_skills'].tolist()
column2_values = match_df[match_df['Corrected -> unique skill'] != '[drop]']['Lightcast_skills'].tolist()

# Create a dictionary where column1 is the key and column2 is the value
semantic_equivalents = dict(zip(column1_values, column2_values))

In [ ]:
# Create a new defaultdict of defaultdict based on semantic equivalents while ensuring symmetry
co_dict_lightcast = defaultdict(lambda: defaultdict(int))

for element1, inner_dict in co_dict_combined.items():
    for element2, count in inner_dict.items():
        # Replace element1 and element2 with their semantic equivalents if they exist
        element1 = semantic_equivalents.get(element1, element1)
        element2 = semantic_equivalents.get(element2, element2)
        
        # Ensure symmetry in the counts
        co_dict_lightcast[element1][element2] += count
        co_dict_lightcast[element2][element1] += count

In [ ]:
len(co_dict_lightcast)

## Convert to Pandas dataframe

In [ ]:
unique_elements = set()
for element1, inner_dict in co_dict_combined.items():
    unique_elements.add(element1)

In [ ]:
# Convert the set of unique elements to a list
unique_elements = list(unique_elements)

# Step 2: Create an empty DataFrame with columns and indices
df = pd.DataFrame(columns=unique_elements, index=unique_elements)

# Step 3: Fill in the DataFrame with values from the defaultdict of defaultdict
for element1, inner_dict in co_dict_combined.items():
    for element2, count in inner_dict.items():
        df.at[element1, element2] = count
        df.at[element2, element1] = count

In [ ]:
df.fillna(0, inplace=True)